<a href="https://colab.research.google.com/github/DarshiniMH/Book-Recommendation-/blob/main/book_rec_User_Item.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rapidfuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 55.4 MB/s eta 0:00:00


In [ ]:
import sqlite3
import pandas as pd
from rapidfuzz import process  # Alternative fuzzy matching in Python


def search_books(title_query, conn, top_n=5):
    """
    Search for a book title using FTS5 + fuzzy matching for typos.

    Args:
    - title_query (str): The user-provided book title.
    - conn (sqlite3.Connection): The database connection.
    - top_n (int): Number of results to return.

    Returns:
    - List of matched book details (book_id, title, ratings_count, average_rating).
    """
    cursor = conn.cursor()

    #  1. Direct FTS5 search (fastest)
    query = f"""
        SELECT b.book_id, b.title, b.ratings_count, b.average_rating
        FROM books_fts fts
        JOIN books b ON b.rowid = fts.rowid
        WHERE books_fts MATCH ?
        ORDER BY b.ratings_count DESC
        LIMIT ?;
    """
    cursor.execute(query, (title_query, top_n))
    results = cursor.fetchall()

    #  2. If no results, apply fuzzy matching using Python's `rapidfuzz`
    if not results:
        print("No exact matches found. Trying fuzzy matching...")

        # Fetch all book titles from the database
        cursor.execute("SELECT title FROM books;")
        all_titles = [row[0] for row in cursor.fetchall()]

        # Use `process.extract` to get the best fuzzy matches
        fuzzy_matches = process.extract(title_query, all_titles, limit=top_n)

        # Get the book details for matched titles
        matched_titles = [match[0] for match in fuzzy_matches]
        query = f"""
            SELECT book_id, title, ratings_count, average_rating
            FROM books
            WHERE title IN ({','.join(['?']*len(matched_titles))})
            ORDER BY ratings_count DESC
            LIMIT ?;
        """
        cursor.execute(query, (*matched_titles, top_n))
        results = cursor.fetchall()

    return results

# Example Usage
conn = sqlite3.connect("/content/drive/MyDrive/Book Recommendation/goodreads_books.db")
search_title = "The alchemist"  # Example with a typo
matched_books = search_books(search_title, conn)

# Display Results
if matched_books:
    print("\n Top Book Matches:")
    for book in matched_books:
        print(f"📖 {book[1]} (Book ID: {book[0]}, Ratings: {book[2]}, Avg Rating: {book[3]})")
else:
    print(" No matches found!")



 Top Book Matches:
📖 The Alchemist (Book ID: 865, Ratings: 1342863, Avg Rating: 3.8200000000000003)
📖 The Neutronium Alchemist (Night's Dawn, #2) (Book ID: 479561, Ratings: 12946, Avg Rating: 4.25)
📖 The Alchemist (Book ID: 25076674, Ratings: 12171, Avg Rating: 3.8200000000000003)
📖 The Alchemist (Book ID: 6071573, Ratings: 8174, Avg Rating: 3.8200000000000003)
📖 The Alchemist (Book ID: 18144590, Ratings: 4908, Avg Rating: 3.8200000000000003)


In [ ]:
!pip install faiss-cpu --no-cache-dir


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 288.3 MB/s eta 0:00:00


In [ ]:
import sqlite3
import pandas as pd

# Connect to the cleaned database
conn = sqlite3.connect("/content/drive/MyDrive/Book Recommendation/goodreads_interactions_cleaned.db")

# Load the cleaned dataset
query = "SELECT user_id, book_id, rating FROM interactions WHERE rating >= 4"
df = pd.read_sql(query, conn)
conn.close()

print(f"Dataset after cleaning: {df.shape}")

Dataset after cleaning: (63755324, 3)


In [ ]:
import sqlite3
import pandas as pd

# Connect to the cleaned database
conn = sqlite3.connect("/content/drive/MyDrive/Book Recommendation/goodreads_interactions.db")

# Load the cleaned dataset
query = "SELECT user_id, book_id, rating FROM interactions_filtered WHERE rating >= 4"
df = pd.read_sql(query, conn)
conn.close()

print(f"Dataset after cleaning: {df.shape}")

Dataset after cleaning: (63125193, 3)


In [ ]:
import sqlite3
import pandas as pd

# ✅ Connect to SQLite database
db_path = "/content/drive/MyDrive/Book Recommendation/goodreads_interactions_cleaned.db"  # Update with your correct path
conn = sqlite3.connect(db_path)
cursor = conn.cursor()


# ✅ Step 2: Fetch Column Names of `interactions` Table
cursor.execute("PRAGMA table_info(interactions);")
columns_info = cursor.fetchall()

# ✅ Extract column names
column_names = [[column[1],column[2]] for column in columns_info]
print(column_names)

# ✅ Close connection
conn.close()

[['user_id', 'INTEGER'], ['book_id', 'INTEGER'], ['rating', 'INTEGER'], ['is_read', 'INTEGER'], ['is_reviewed', 'INTEGER']]


In [ ]:
import sqlite3
import pandas as pd
from scipy.sparse import coo_matrix
import numpy as np
import faiss
import os

# Define chunk size
chunk_size = 500000

#  Connect to the SQLite database (Ensure `book_id` is an INTEGER)
conn = sqlite3.connect("/content/drive/MyDrive/Book Recommendation/goodreads_interactions.db")

#  Create empty lists to store chunks
user_indices = []
book_indices = []
ratings = []

#  Read data in chunks
query = "SELECT user_id, book_id, rating FROM interactions_filtered WHERE rating > 3"
chunks = pd.read_sql_query(query, conn, chunksize=chunk_size)

for chunk in chunks:
    # Ensure `book_id` and `user_id` remain INTEGER
    chunk["user_index"] = chunk["user_id"].astype("category").cat.codes
    chunk["book_index"] = chunk["book_id"].astype("category").cat.codes
    chunk["rating"] = pd.to_numeric(chunk["rating"], downcast="integer")

    # Store chunk data in lists
    user_indices.extend(chunk["user_index"].tolist())
    book_indices.extend(chunk["book_index"].tolist())
    ratings.extend(chunk["rating"].tolist())

#  Close database connection
conn.close()

#  Create a sparse book-user interaction matrix
book_user_matrix = coo_matrix((ratings, (book_indices, user_indices))).tocsr()



In [ ]:
#  Perform Dimensionality Reduction (Truncated SVD)
from sklearn.decomposition import TruncatedSVD

n_books, n_users = book_user_matrix.shape
print("Original dimensionality:", n_users)

# Retain 98% variance
target_variance = 0.98
svd = TruncatedSVD(n_components=min(n_users,1000), random_state=42)
reduced_matrix = svd.fit_transform(book_user_matrix)
explained = svd.explained_variance_ratio_.sum()
print(f"Explained variance with {min(n_users,1000)} components: {explained*100:.2f}%")


#  Find optimal number of dimensions
cumulative_variance = np.cumsum(svd.explained_variance_ratio_)
optimal_d = np.searchsorted(cumulative_variance, target_variance) + 1
reduced_vectors = reduced_matrix[:, :optimal_d].astype("float32")
print(f"Reduced dimensionality: {optimal_d}")

Original dimensionality: 8202
Explained variance with 1000 components: 63.56%
Reduced dimensionality: 1001


In [ ]:
#  Normalize vectors for cosine similarity
def normalize_vectors(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    return vectors / norms

reduced_vectors = normalize_vectors(reduced_vectors)

#  Create FAISS Index for Fast Similarity Search
nlist = 4096  # Number of Voronoi clusters
m = 16        # PQ sub-quantizers
nbits = 8     # Bits per sub-vector
optimal_d = int((optimal_d // m) * m)  # Round down to nearest multiple of m

reduced_vectors = reduced_vectors[:, :optimal_d]

#  Use FAISS Index for Cosine Similarity
quantizer = faiss.IndexFlatIP(optimal_d)
index = faiss.IndexIVFPQ(quantizer, optimal_d, nlist, m, nbits)
print("Cosine similarity index created with dimension", optimal_d)

#  Train FAISS index
sample_size = min(100000, n_books)
train_data = reduced_vectors[np.random.choice(n_books, sample_size, replace=False)]
index.train(train_data)
print("Index trained on sample of size:", sample_size)

#  Add vectors to FAISS in batches
batch_size = 50000
for i in range(0, n_books, batch_size):
    end = min(i + batch_size, n_books)
    index.add(reduced_vectors[i:end])
    print(f"Added vectors {i} to {end}")

print("Total indexed vectors:", index.ntotal)


In [ ]:
#  Save FAISS index to disk
faiss.write_index(index, "/content/drive/MyDrive/Book Recommendation/books_index.faiss")
print("Index saved to books_index.faiss (size: %.2f MB)" % (os.path.getsize("/content/drive/MyDrive/Book Recommendation/books_index.faiss") / 1024**2))

#  Load FAISS index for querying
index = faiss.read_index("/content/drive/MyDrive/Book Recommendation/books_index.faiss")
index.nprobe = 10
print("Index loaded. Total vectors:", index.ntotal)

# 3.3 Example query: find top-5 similar books for a given book vector (or user vector)
query_vec = reduced_vectors[0]  # e.g., first book's vector as a query
D, I = index.search(query_vec.reshape(1, -1), k=5)
print("Recommended book IDs for the query:", I[0])

In [ ]:
def get_recommendations(book_id, book_mapping, reduced_vectors, index, k=5):
    """
    Get top-k similar books for a given book_id.

    Args:
    - book_id (int): The book_id from the dataset.
    - book_mapping (dict): Mapping from book_id to book_index.
    - reduced_vectors (numpy array): The book embedding matrix.
    - index (faiss index): The trained FAISS index.
    - k (int): Number of recommendations to retrieve.

    Returns:
    - recommended_books (list): List of recommended books with their details.
    """
    conn = sqlite3.connect("/content/drive/MyDrive/Book Recommendation/goodreads_books.db")
    cursor = conn.cursor()

    # ✅ Step 1: Ensure book_id is valid
    if book_id not in book_mapping:
        print(f"Error: Book ID {book_id} not found in mapping.")
        return []

    book_index = book_mapping[book_id]

    # ✅ Step 2: Retrieve book vector
    query_vec = reduced_vectors[book_index].reshape(1, -1)

    # ✅ Step 3: Perform FAISS search
    D, I = index.search(query_vec, k+1)

    # ✅ Step 4: Convert book indices back to book_ids
    recommended_book_ids = [key for key, val in book_mapping.items() if val in I[0] and val != book_index]

    # ✅ Step 5: Fetch book details from `goodreads_books.db`
    placeholders = ",".join(["?"] * len(recommended_book_ids))
    query = f"SELECT book_id, title, average_rating FROM books WHERE book_id IN ({placeholders}) ORDER BY average_rating DESC"
    cursor.execute(query, recommended_book_ids)
    recommended_books = cursor.fetchall()

    conn.close()

    return recommended_books
"""
# ✅ Example: Get recommendations for a book
book_id_to_query = 5470  # Replace with a real book_id from your dataset
book_mapping = {book_id: idx for idx, book_id in enumerate(pd.read_sql("SELECT book_id FROM books", sqlite3.connect("/content/drive/MyDrive/Book Recommendation/goodreads_books.db"))["book_id"])}
recommended_books = get_recommendations(book_id_to_query, book_mapping, reduced_vectors, index, k=5)

# ✅ Display Results
if recommended_books:
    print("\n📚 Top 5 Recommended Books:")
    for book in recommended_books:
        print(f"📖 {book[1]} (Book ID: {book[0]}, Avg Rating: {book[2]})")
else:
    print("❌ No recommendations found!")
    """


'\n# ✅ Example: Get recommendations for a book\nbook_id_to_query = 5470  # Replace with a real book_id from your dataset\nbook_mapping = {book_id: idx for idx, book_id in enumerate(pd.read_sql("SELECT book_id FROM books", sqlite3.connect("/content/drive/MyDrive/Book Recommendation/goodreads_books.db"))["book_id"])}\nrecommended_books = get_recommendations(book_id_to_query, book_mapping, reduced_vectors, index, k=5)\n\n# ✅ Display Results\nif recommended_books:\n    print("\n📚 Top 5 Recommended Books:")\n    for book in recommended_books:\n        print(f"📖 {book[1]} (Book ID: {book[0]}, Avg Rating: {book[2]})")\nelse:\n    print("❌ No recommendations found!")\n    '

In [ ]:
# ✅ Connect to the books database
conn = sqlite3.connect("/content/drive/MyDrive/Book Recommendation/goodreads_books.db")

# ✅ Step 1: Search for the book by title
search_title = "The alchemist"  # Example search query
matched_books = search_books(search_title, conn)  # Calls the search function

# ✅ Step 2: Get the first book_id from the search results
if matched_books:
    first_book_id = matched_books[0][0]  # Extract the book_id of the first match

    print(f"\n📖 Selected Book for Recommendation: {matched_books[0][1]} (Book ID: {first_book_id})")

    # ✅ Step 3: Load the book_id → book_index mapping
    book_mapping_conn = sqlite3.connect("/content/drive/MyDrive/Book Recommendation/goodreads_books.db")
    book_mapping_query = "SELECT book_id FROM books"
    book_mapping_df = pd.read_sql(book_mapping_query, book_mapping_conn)
    book_mapping = {book_id: idx for idx, book_id in enumerate(book_mapping_df["book_id"])}

    # ✅ Step 4: Call get_recommendations() with the retrieved book_id
    recommended_books = get_recommendations(first_book_id, book_mapping, reduced_vectors, index, k=5)

    # ✅ Step 5: Display the recommended books
    if recommended_books:
        print("\n📚 Recommended Books:")
        for book in recommended_books:
            print(f"📖 {book[1]} (Book ID: {book[0]}, Avg Rating: {book[2]})")
    else:
        print("❌ No recommendations found!")
else:
    print("❌ No matches found for the given book title!")


📖 Selected Book for Recommendation: The Alchemist (Book ID: 865)

📚 Recommended Books:
📖 J.R.R. Tolkien 4-Book Boxed Set: The Hobbit and The Lord of the Rings (Book ID: 30, Avg Rating: 4.59)
📖 Harry Potter and the Half-Blood Prince (Harry Potter, #6) (Book ID: 1, Avg Rating: 4.54)
📖 The Lord of the Rings (The Lord of the Rings, #1-3) (Book ID: 32, Avg Rating: 4.48)
📖 Harry Potter and the Sorcerer's Stone (Harry Potter, #1) (Book ID: 3, Avg Rating: 4.45)
📖 The Mother Tongue: English and How It Got That Way (Book ID: 29, Avg Rating: 3.95)
